In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
import numpy as np

In [15]:
url ='https://www.zomato.com/bangalore/'
driver = webdriver.Firefox()
driver.get(url)

In [6]:
soup = BeautifulSoup(driver.page_source,"html.parser")

In [7]:
locationlist = []
for link in soup.find_all('a', href = True):
    if 'https://www.zomato.com/bangalore/' in link['href']:
        locationlist.append(link['href'])
print(len(locationlist))

29


In [16]:
link = []
for i in locationlist:
    driver.get(i)
    driver.maximize_window()
    driver.execute_script("window.scrollBy(0,2500)","")
    time.sleep(10)
    
    links = driver.find_elements('tag name','a')
    for i in links:
        link.append(i.get_attribute('href'))

In [17]:
href=[]
for i in link:
    try:
        if 'order' in i:
            href.append(i)
    except:
        pass

In [18]:
final_href = set(href)
print(len(final_href))

134


In [ ]:
href_new=[]
for i in final_href:
    try:
        if 'order' in i:
            href_new.append(i)
    except:
        pass

In [ ]:
print(len(href_new))

In [ ]:
name=[]
rating=[]
cusines=[]
location=[]
delivery_review_number=[]
count = 0
for i in href_new:
    count+=1
    print(count,end=" ")
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    r=requests.get(i,headers=headers).text
    sou = BeautifulSoup(r, 'lxml')
    try:
        j = sou.find_all("h1",{"class" : "sc-7kepeu-0 sc-eilVRo eAhpQG"})
        name.append(j[0].text)
    except:
        name.append('NA')
    try:
        j = sou.find_all("div",{"class" : "sc-1q7bklc-1 cILgox"})
        rating.append(j[1].text)
    except:
        rating.append('NA')
    
    try:
        j = sou.find_all("div",{"class" : "sc-eMigcr fAGAHS"})
        cusines.append(j[0].text)
    except:
        cusines.append('NA')
    try:
        j = sou.find_all("a",{"class" : "sc-cpmLhU fDVcNc"})
        location.append(j[0].text)
    except:
        location.append('NA')

    try:
        j = sou.find_all("div",{"class" : "sc-1q7bklc-8 kEgyiI"})
        delivery_review_number.append(j[1].text)
    except:
        delivery_review_number.append('NA')
    
    

In [ ]:
col=['name','rating','cusines','delivery_review_number','location']
df=pd.DataFrame({'Name': name,'Rating':rating,'cusines':cusines,'delivery_review_number':delivery_review_number,'location':location})
df

In [ ]:
longitude = []
latitude = []
   
# function to find the coordinate
# of a given city 
def findGeocode(location): 
    try:
        geolocator = Nominatim(user_agent="chrome")
          
        return geolocator.geocode(location)
      
    except GeocoderTimedOut:
          
        return findGeocode(location)  
for i in (df["location"]):
      
    if findGeocode(i) != None:
           
        loc = findGeocode(i)
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)

    else:
        latitude.append(np.nan)
        longitude.append(np.nan)

In [ ]:
df["Longitude"] = longitude
df["Latitude"] = latitude

In [ ]:
df["Restaurant_link"]  = href_new

In [ ]:
df.to_csv(r"C:\Users\Vaishali Tomar\Desktop\resume\zomato_1.csv")

In [ ]:
restaurant_name = []
dish_name = []
dish_price = []

count = 0

for i in href_new :
    count+=1
    print(count,end=" ")
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    r=requests.get(i,headers=headers).text
    sou = BeautifulSoup(r, 'lxml')
    for i in range(10):
        try:
            j = sou.find_all("h4",{"class" : "sc-1s0saks-15 iSmBPS"})
            dish_name.append(j[i].text)
        except:
            dish_name.append('NA')
        try:
            j = sou.find_all("span",{"class" : "sc-17hyc2s-1 cCiQWA"})
            dish_price.append(j[i].text)
        except:
            dish_price.append('NA')
        try:
            j = sou.find_all("h1",{"class" : "sc-7kepeu-0 sc-eilVRo eAhpQG"})#sc-7kepeu-0 sc-eilVRo eAhpQG
            restaurant_name.append(j[0].text)
        except:
            restaurant_name.append('NA')

        


In [ ]:
col=['restaurant_name','dish_name','cusines','dish_price']
df1=pd.DataFrame({'restaurant_name': restaurant_name,'dish_name':dish_name,'dish_price':dish_price})
df1

In [ ]:
df1.to_csv(r"C:\Users\Vaishali Tomar\Desktop\resume\zomato_2.csv")